In [1]:
##This notebook trains the same CNN model, but with l1 regularization to encourage smaller and sparser weights
##We use 10^-3 learning rate instead of 10^-4, we also let the model run longer. After 50000 batches of size 50 
##(~41 epochs) the l1 norm of the weights was 28000, compared to 230000 for the baseline model

import mnist_input
import mnist_model
reload(mnist_model)
import numpy as np
import tensorflow as tf
mnist = mnist_input.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
checkpoint_path = "/home/justin/Programming/AdverserialMNIST/saved_models/l1reg.ckpt"
x, y_ = mnist_model.place_holders()
y_conv, keep_prob, variable_dict = mnist_model.model(x)

In [11]:
#define the l1-loss
absolute_sums = []
for variable in variable_dict.values():
    absolute_sums.append(tf.reduce_sum(tf.abs(variable)))
l1_sum = tf.add_n(absolute_sums)

#count the number where are > 0, surprisingly 
variable_non_zero_counts = []
for variable in variable_dict.values():
    variable_non_zero_counts.append(tf.reduce_sum(tf.cast(tf.greater(tf.abs(variable), tf.constant(0.000001)), "float")))
num_non_zero_op = tf.add_n(variable_non_zero_counts)

In [4]:
#experiments without l1 regularization showed cross entropy in the range of [1,10] and
#l1_sum around 236000
C = 1.0/20000.0 #define the regularization constant, this way
cross_entropy = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(y_conv,1e-10,1.0))) #avoid 0*log(0) error
l1_loss = cross_entropy + C*l1_sum

train_step = tf.train.AdamOptimizer(1e-3).minimize(l1_loss)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
saver = tf.train.Saver(variable_dict)
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
#saver.restore(sess, checkpoint_path)

In [5]:
def get_test_accuracy(prob = 1.0):
    idx = 0
    batch_size = 500
    num_correct = 0
    while(idx < len(mnist.test.images)):
        num_correct += np.sum(correct_prediction.eval(feed_dict = {
               x: mnist.test.images[idx:idx+batch_size], 
               y_: mnist.test.labels[idx:idx+batch_size], keep_prob: prob
                    }))
        idx+=batch_size
    return float(num_correct)/float(len(mnist.test.images))

In [13]:
for i in range(50000):
  batch = mnist.train.next_batch(50)
  if(i%1000 == 0):
    saver.save(sess, checkpoint_path)
    cross_ent, l1_norm, train_accuracy, loss_func, num_non_zero = sess.run(
        [cross_entropy, l1_sum, accuracy, l1_loss, num_non_zero_op], 
                                  feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    test_accuracy = get_test_accuracy()
    print "step %d, cross_ent: %f, l1_norm: %f, train_acc: %f, test_acc: %f, loss: %f, num_non_zero: %d" \
            %(i, cross_ent, l1_norm, train_accuracy, test_accuracy, loss_func, num_non_zero)

  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

#print("test accuracy %g"%accuracy.eval(feed_dict={
#    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))



step 0, cross_ent: 0.096167, l1_norm: 31424.892578, train_acc: 1.000000, test_acc: 0.992100, loss: 1.667411, num_non_zero: 1618356
step 1000, cross_ent: 0.002629, l1_norm: 31625.417969, train_acc: 1.000000, test_acc: 0.990700, loss: 1.583899, num_non_zero: 1618353
step 2000, cross_ent: 0.049666, l1_norm: 30756.298828, train_acc: 1.000000, test_acc: 0.992200, loss: 1.587481, num_non_zero: 1618899
step 3000, cross_ent: 0.001304, l1_norm: 31521.630859, train_acc: 1.000000, test_acc: 0.993200, loss: 1.577386, num_non_zero: 1619387
step 4000, cross_ent: 0.000730, l1_norm: 29586.613281, train_acc: 1.000000, test_acc: 0.992000, loss: 1.480060, num_non_zero: 1619878
step 5000, cross_ent: 0.002505, l1_norm: 28110.949219, train_acc: 1.000000, test_acc: 0.991600, loss: 1.408052, num_non_zero: 1619748
step 6000, cross_ent: 0.023622, l1_norm: 30574.423828, train_acc: 1.000000, test_acc: 0.991900, loss: 1.552343, num_non_zero: 1613089
step 7000, cross_ent: 0.009392, l1_norm: 31347.361328, train_acc:

KeyboardInterrupt: 

In [14]:
print get_test_accuracy()

0.9922
